# Imports

In [1]:
from numpy.random import seed
seed(888)
import tensorflow
tensorflow.random.set_seed(404)

In [20]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

from time import strftime
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# Get the Data

In [4]:
%%time

x_train = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: user 42.8 s, sys: 1.3 s, total: 44.1 s
Wall time: 57.1 s


In [5]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

# Explore

In [6]:
x_train.shape

(60000, 784)

In [7]:
y_test.shape

(10000,)

# Data Preprocessing

In [8]:
# Re-scale
x_train = x_train/ 255.0, y_test / 255.0

#### Convert target values to one-hot encoding

In [10]:
values = y_test[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [11]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [12]:
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
values

array([7, 2, 1, 0, 4])

In [14]:
values[4]

4

In [15]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [43]:
y_val = y_test[:VALIDATION_SIZE]


# Setup Tensorflow Graph

In [21]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

#### Hyperparameters

In [22]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [23]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [ ]:
# Model without dropout
# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
#                       bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], 
#                       bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
#                       bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [24]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Tensorboard Setup

In [25]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss, Optimisation & Metrics

#### Defining Loss Function

In [26]:

with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [27]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric

In [28]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [29]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [30]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [31]:
sess = tf.Session()

#### Setup Filewriter and Merge Summaries

In [32]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

#### Initialise all the variables

In [33]:

init = tf.global_variables_initializer()
sess.run(init)

### Batching the Data

In [34]:
size_of_batch = 1000

In [37]:
num_examples = y_test.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [38]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

### Training Loop

In [51]:
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ======================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

ValueError: Cannot feed value of shape (0,) for Tensor 'X:0', which has shape '(?, 784)'

# Make a Prediction

In [ ]:
img = Image.open('MNIST/test_img.png')
img

In [ ]:
bw = img.convert('L')

In [ ]:
img_array = np.invert(bw)

In [ ]:
img_array.shape

In [ ]:
test_img = img_array.ravel()

In [ ]:
test_img.shape

In [ ]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})

In [ ]:
print(f'Prediction for test image is {prediction}')

# Testing and Evaluation

In [ ]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

# Reset for the Next Run

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

# Code for 1st Part of Module

In [ ]:
# with tf.name_scope('hidden_1'):

#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1

#     layer1_out = tf.nn.relu(layer1_in)

In [ ]:
# with tf.name_scope('hidden_2'):

#     initial_w2 = tf.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [ ]:
# with tf.name_scope('output_layer'):

#     initial_w3 = tf.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [ ]:
# b3.eval(sess)